In [1]:
import pandas as pd
import gensim
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import plotly.plotly as py
import plotly
import cufflinks as cf
import pprint
import json
import yaml

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.manifold import TSNE





paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress


In [40]:
# main_data = pd.read_csv('../../business_news.csv')
# main_data = pd.read_csv('../../entrepreneurs_news.csv')
# main_data = pd.read_csv('../../financial_news.csv')

main_data = main_data.sort_values(by = "Published")

news_data = main_data[["Snippet", "Lead Paragraph", "Main Headline"]]
news_data.apply(str)
main_data.head()

,Published,Snippet,Lead Paragraph,Main Headline,Print Headline,Seo Headline
977,2019-01-01T07:30:05+0000,Underperforming media deals have left the conf...,Underperforming media deals have left the conf...,Another Season Comes and Goes While Pac-12 Str...,A Power Conference’s Declining Clout,NaN
674,2019-01-01T14:00:04+0000,Recent commercial real estate transactions in ...,Recent commercial real estate transactions in ...,Recent Commercial Real Estate Transactions,Transactions,NaN
879,2019-01-01T17:47:14+0000,Skeptical reporting has still been too favorable.,Skeptical reporting has still been too favorable.,The Trump Tax Cut: Even Worse Than You’ve Heard,The Trump Tax Cut: Even Worse Than You’ve Heard,NaN
307,2019-01-01T21:21:40+0000,"The episode of “Patriot Act With Hasan Minhaj,...","The episode of “Patriot Act With Hasan Minhaj,...",Netflix Blocks Show in Saudi Arabia Critical o...,Netflix Blocks Show in Saudi Arabia Critical o...,NaN
357,2019-01-02T07:00:00+0000,This word has appeared in 10 articles on NYTim...,This word has appeared in 10 articles on NYTim...,Word + Quiz: agog,Word + Quiz: agog,NaN


In [41]:
def list_prep(rows_to_list):
    news_list = rows_to_list.values.tolist()
    return(news_list)
    
list_of_lists = list_prep(news_data)
print(list_of_lists[0])



['Underperforming media deals have left the conference falling behind its peers off the field. That might be the root of its problems on it.', 'Underperforming media deals have left the conference falling behind its peers off the field. That might be the root of its problems on it.', 'Another Season Comes and Goes While Pac-12 Struggles to Keep Up']


In [42]:
def word_prep(list_lists):
    big_string = ""

    for title in list_lists:
    #     Splitting titles into list of words
        variable = title.split(" ")
    #     print(variable)

    #     v = words
    #     variable = list of words
        for v in variable:
            big_string = big_string + " " + v

    #     big_title_string = ' '.join((str(v) for v in variable))

    tokens = word_tokenize(big_string)

    # Remove non-alphabetic tokens, such as punctuation
    words = [word.lower() for word in tokens if word.isalpha()]

    # Filter out stopwords
    stop_words = set(stopwords.words('english'))

    words = [word for word in words if not word in stop_words]

    return(words)


    # print(tokens)
word_list = []
for list in list_of_lists:
    words = word_prep(list)
    word_list.append(words)
print(word_list[0])

['underperforming', 'media', 'deals', 'left', 'conference', 'falling', 'behind', 'peers', 'field', 'might', 'root', 'problems', 'underperforming', 'media', 'deals', 'left', 'conference', 'falling', 'behind', 'peers', 'field', 'might', 'root', 'problems', 'another', 'season', 'comes', 'goes', 'struggles', 'keep']


In [8]:
# Load word2vec model (trained on an enormous Google corpus)
model = gensim.models.KeyedVectors.\
load_word2vec_format('../resources/model/GoogleNews-vectors-negative300.bin', binary = True) 

# Check dimension of word vectors
model.vector_size


300

In [43]:
def word_2_vec(words):
    vector_list = []
    words_filtered = []
    # Filter the list of vectors to include only those that Word2Vec has a vector for
    vector_list.extend([model[word] for word in words if word in model.vocab])
#     return(vector_list)
    # Create a list of the words corresponding to these vectors
    words_filtered.extend([word for word in words if word in model.vocab])
#     return(words_filtered)
    # Zip the words together with their vector representations
    word_vec_zip = zip(words_filtered, vector_list)
    
    # Cast to a dict so we can turn it into a dataframe
    word_vec_dict = dict(word_vec_zip)
    return(word_vec_dict)
    


dataframe_dict_list = []
for list in word_list:
    df = word_2_vec(list)
    dataframe_dict_list.append(df)


In [44]:

dataframe_list = []
for dataframe in range(len(dataframe_dict_list)):
    df = pd.DataFrame.from_dict(dataframe_dict_list[dataframe])
    dataframe_list.append(df)
# df.shape
dataframe_list[0].head()

,underperforming,media,deals,left,conference,falling,behind,peers,field,might,root,problems,another,season,comes,goes,struggles,keep
0,-0.166992,0.097656,0.105957,0.233398,-0.146484,-0.085449,0.125977,-0.275391,-0.294922,0.227539,-0.116211,-0.023560,0.194336,0.059814,0.265625,0.090332,0.378906,0.060547
1,0.238281,-0.009277,0.337891,0.073242,0.118164,0.241211,-0.014709,0.074707,0.146484,0.120117,0.243164,0.163086,-0.019653,0.164062,0.037842,0.148438,0.289062,-0.012939
2,0.291016,-0.267578,-0.088379,0.044434,0.242188,-0.116211,0.145508,-0.164062,0.123047,0.068359,0.163086,-0.114258,0.091797,-0.159180,0.069336,-0.061768,-0.208008,-0.108887
3,-0.107422,-0.281250,-0.022827,-0.010254,-0.076660,-0.001678,-0.158203,0.351562,-0.292969,0.324219,0.130859,0.273438,0.104492,0.061523,-0.029419,0.059082,0.132812,0.143555
4,-0.135742,0.016113,0.028809,-0.055664,0.062988,-0.033691,-0.106445,0.134766,0.034668,-0.161133,-0.527344,-0.236328,-0.071777,0.197266,-0.083984,-0.013184,0.004395,-0.087402


In [45]:
def row_summation(dataframe_list):
    summation_list = []
    for dataframe in dataframe_list:
    #     print(dataframe)
        total_columns = len(dataframe.columns)
    #     print(total_columns)
        sums = dataframe.sum(axis = 1, skipna = True)
    #     print(sums)
        summation = sums/total_columns
        summation_list.append(summation)
    return summation_list

    
summation_series = row_summation(dataframe_list)
print(summation_series[0][:5])

0    0.040616
1    0.129954
2   -0.002753
3    0.033381
4   -0.057427
dtype: float32


In [46]:
published_list = main_data['Published'].tolist()

In [47]:
summation_dict = {}
published_index = 0
for series in summation_series:
    summation_dict.update({published_list[published_index]: series})
    published_index += 1
    
summation_df = pd.DataFrame(summation_dict)
summation_df = summation_df.T
summation_df.head()




,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
2019-01-01T07:30:05+0000,0.040616,0.129954,-0.002753,0.033381,-0.057427,-0.013980,0.042677,-0.091292,0.117317,0.057339,...,-0.050925,0.051815,-0.130132,0.007858,-0.025178,0.005754,0.025302,-0.020338,0.020962,0.002306
2019-01-01T14:00:04+0000,-0.006548,0.075945,-0.091047,0.003771,0.058803,-0.076625,-0.049800,-0.061550,0.137486,0.082084,...,-0.053502,0.019948,-0.003523,0.008057,0.019845,-0.006598,-0.036187,-0.053292,-0.040545,-0.105887
2019-01-01T17:47:14+0000,0.051382,0.038457,-0.040468,0.091321,-0.105566,0.050043,0.067348,0.034418,0.221899,0.109424,...,0.018445,0.025484,0.005743,0.020642,-0.013770,-0.054761,-0.026877,-0.051270,0.094012,-0.003528
2019-01-01T21:21:40+0000,-0.027873,0.047825,0.052103,0.020469,-0.090712,0.011233,0.030041,-0.041097,0.083566,0.067819,...,-0.020653,-0.104341,-0.046093,-0.068028,-0.011126,0.021588,-0.013541,-0.066595,0.059064,0.035169
2019-01-02T07:00:00+0000,0.144113,0.129674,-0.027830,0.098580,-0.037144,-0.040632,0.109305,-0.168248,0.135254,0.042855,...,0.019758,-0.118478,-0.038942,-0.057478,0.082912,-0.200335,-0.007311,-0.116525,-0.052213,-0.024972


In [48]:
summation_df.to_csv("../resources/news/financial_news_word2vec.csv")